In [7]:
import yaml
from run_experiment import choose_dataset_class,choose_task_classes

import torch
import numpy as np
from collections import Counter

In [9]:
# 0. read the embeddings and labesl
yaml_args= yaml.load(open('../SA-config/a-sample-pos-get_cor.yaml'))

dataset_class = choose_dataset_class(yaml_args)

task_class, reporter_class, loss_class = choose_task_classes(yaml_args)

task = task_class(yaml_args)

expt_dataset = dataset_class(yaml_args, task)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


Loading BERT Pretrained Embeddings from ../example/data/en_ewt-ud/en_ewt-ud-train.bert-base-layers.hdf5; using layer 1


[aligning embeddings]:   0%|          | 7/12543 [00:00<03:45, 55.55it/s]

Using BERT-base-cased tokenizer to align embeddings with PTB tokens


[aligning embeddings]: 100%|██████████| 12543/12543 [02:54<00:00, 71.90it/s] 
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


...

[aligning embeddings]:   1%|          | 11/2077 [00:00<00:18, 109.87it/s]

Using BERT-base-cased tokenizer to align embeddings with PTB tokens


[computing labels]:   3%|▎         | 351/12543 [00:00<00:03, 3502.96it/s]

CORRUPTED 0
Retaining 12543 training observations


[computing labels]: 100%|██████████| 2077/2077 [00:00<00:00, 7065.23it/s]


In [10]:
# 1. prepare the label matrix.
train_data = expt_dataset.train_dataset
labels=[]
for obser in train_data.observations:
    labels.append(task.labels(obser))

all_labels = torch.cat(labels, 0).numpy().astype(int)

all_labels.shape
all_labels_mat= np.zeros((all_labels.size, all_labels.max()+1))
all_labels_mat[np.arange(all_labels.size),all_labels] = 1
all_labels_mat.shape

(204609, 50)

In [11]:
# 2. prepare the embedding matrix.

embeddings=[]
for obser in train_data.observations:
    embeddings.append(obser.embeddings)

all_embeddings = torch.cat(embeddings, 0).numpy()

all_embeddings.shape

(204609, 768)

In [12]:
# 3. compute the corrcoef matrix. 
# num=204609
# corr_mat_whole = np.corrcoef(all_labels_mat[0:num].T, all_embeddings[0:num].T)
corr_mat_whole = np.corrcoef(all_labels_mat.T, all_embeddings.T)
corr_mat=np.absolute(corr_mat_whole[0:50,50:])
corr_mat.shape

(50, 768)

In [ ]:
# 4. output the corr_mat data. dump as a pkl.


In [13]:
# 4. output the corr_sum_dim
# we compute the corr score of each dim for all the 50 pos labels.
dim_sum=[]
for i in range(768):
    dim_sum.append(np.sum(corr_mat[:,i]))

corr_sum_dim=np.argsort(dim_sum)[::-1]
with open('../SA-dim-files/average_corr_dim.tsv','w') as fout:
    for dim in corr_sum_dim:
        fout.write('{} '.format(dim))

In [14]:
# 5. output the weighted_average_corr_dim
# we compute the weighted average corr socre of each dim for all the pos labels

# 5.1 get the distribution/weights 
all_labels
result = Counter(all_labels)
weights = [result[i]/len(all_labels) for i in range(50)]



# 5.2 get the weighted score
dim_weighted_average=[]
for i in range(768):
    dim_weighted_average.append(np.average(corr_mat[:,i],weights=weights))
    
# 5.3 output the dims
dim_weighted_average_dim=np.argsort(dim_weighted_average)[::-1]
with open('../SA-dim-files/weighted_average_corr_dim.tsv','w') as fout:
    for dim in dim_weighted_average_dim:
        fout.write('{} '.format(dim))

In [15]:
# 6. output the max_corr
max_val=[]
for i in range(768):
    max_val.append(np.max(corr_mat[:,i]))

dim_max_val=np.argsort(max_val)[::-1]
with open('../SA-dim-files/max_corr_dim.tsv','w') as fout:
    for dim in dim_weighted_average_dim:
        fout.write('{} '.format(dim))

In [16]:
np.max(dim_sum),np.max(dim_weighted_average),np.max(corr_mat),np.max(max_val)

(4.1708928549625375,
 0.19993299987706492,
 0.4437054850588246,
 0.4437054850588246)

In [17]:

def make_corr_file(layer=1):

    # 0. read the embeddings and labesl
    yaml_args= yaml.load(open('../SA-config/a-sample-pos-get_cor.yaml'))

    yaml_args['model']['model_layer']=layer-1

    dataset_class = choose_dataset_class(yaml_args)

    task_class, reporter_class, loss_class = choose_task_classes(yaml_args)

    task = task_class(yaml_args)

    expt_dataset = dataset_class(yaml_args, task)

    # 1. prepare the label matrix.
    train_data = expt_dataset.train_dataset
    labels=[]
    for obser in train_data.observations:
        labels.append(task.labels(obser))

    all_labels = torch.cat(labels, 0).numpy().astype(int)

    all_labels.shape
    all_labels_mat= np.zeros((all_labels.size, all_labels.max()+1))
    all_labels_mat[np.arange(all_labels.size),all_labels] = 1
    all_labels_mat.shape

    # 2. prepare the embedding matrix.

    embeddings=[]
    for obser in train_data.observations:
        embeddings.append(obser.embeddings)

    all_embeddings = torch.cat(embeddings, 0).numpy()

    all_embeddings.shape

    # 3. compute the corrcoef matrix. 
    # num=204609
    # corr_mat_whole = np.corrcoef(all_labels_mat[0:num].T, all_embeddings[0:num].T)
    corr_mat_whole = np.corrcoef(all_labels_mat.T, all_embeddings.T)
    corr_mat=np.absolute(corr_mat_whole[0:50,50:])
    corr_mat.shape

    # 4. output the corr_mat data. dump as a pkl.


    # 4. output the corr_sum_dim
    # we compute the corr score of each dim for all the 50 pos labels.
    dim_sum=[]
    for i in range(768):
        dim_sum.append(np.sum(corr_mat[:,i]))

    corr_sum_dim=np.argsort(dim_sum)[::-1]
    with open('../SA-dim-files/average_corr_dim_layer_{}.tsv'.format(layer),'w') as fout:
        for dim in corr_sum_dim:
            fout.write('{} '.format(dim))

    # 5. output the weighted_average_corr_dim
    # we compute the weighted average corr socre of each dim for all the pos labels

    # 5.1 get the distribution/weights 
    all_labels
    result = Counter(all_labels)
    weights = [result[i]/len(all_labels) for i in range(50)]



    # 5.2 get the weighted score
    dim_weighted_average=[]
    for i in range(768):
        dim_weighted_average.append(np.average(corr_mat[:,i],weights=weights))

    # 5.3 output the dims
    dim_weighted_average_dim=np.argsort(dim_weighted_average)[::-1]
    with open('../SA-dim-files/weighted_average_corr_dim_layer_{}.tsv'.format(layer),'w') as fout:
        for dim in dim_weighted_average_dim:
            fout.write('{} '.format(dim))

    # 6. output the max_corr
    max_val=[]
    for i in range(768):
        max_val.append(np.max(corr_mat[:,i]))

    dim_max_val=np.argsort(max_val)[::-1]
    with open('../SA-dim-files/max_corr_dim_layer_{}.tsv'.format(layer),'w') as fout:
        for dim in dim_weighted_average_dim:
            fout.write('{} '.format(dim))

In [18]:
make_corr_file(layer=1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


Loading BERT Pretrained Embeddings from ../example/data/en_ewt-ud/en_ewt-ud-train.bert-base-layers.hdf5; using layer 0


[aligning embeddings]:   0%|          | 34/12543 [00:00<00:36, 338.81it/s]

Using BERT-base-cased tokenizer to align embeddings with PTB tokens


[aligning embeddings]: 100%|██████████| 12543/12543 [00:29<00:00, 421.56it/s]
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


...

[aligning embeddings]:   2%|▏         | 42/2077 [00:00<00:04, 417.28it/s]

Using BERT-base-cased tokenizer to align embeddings with PTB tokens


[computing labels]:   3%|▎         | 350/12543 [00:00<00:03, 3494.16it/s]

CORRUPTED 0
Retaining 12543 training observations


[computing labels]: 100%|██████████| 2077/2077 [00:00<00:00, 7565.72it/s]


In [19]:
make_corr_file(layer=3)
make_corr_file(layer=6)
make_corr_file(layer=9)
make_corr_file(layer=12)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


Loading BERT Pretrained Embeddings from ../example/data/en_ewt-ud/en_ewt-ud-train.bert-base-layers.hdf5; using layer 2


[aligning embeddings]:   0%|          | 30/12543 [00:00<00:42, 292.45it/s]

Using BERT-base-cased tokenizer to align embeddings with PTB tokens


[aligning embeddings]: 100%|██████████| 12543/12543 [00:29<00:00, 421.01it/s]
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


...

[aligning embeddings]:   0%|          | 8/2077 [00:00<00:33, 62.13it/s]

Using BERT-base-cased tokenizer to align embeddings with PTB tokens


[computing labels]:   3%|▎         | 336/12543 [00:00<00:03, 3356.18it/s]

CORRUPTED 0
Retaining 12543 training observations


[computing labels]: 100%|██████████| 2077/2077 [00:00<00:00, 7230.32it/s]


In [ ]:
for layer in [1,2,4,5,7,8,10,11]:
    make_corr_file(layer=layer)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


Loading BERT Pretrained Embeddings from ../example/data/en_ewt-ud/en_ewt-ud-train.bert-base-layers.hdf5; using layer 0


[aligning embeddings]:   0%|          | 34/12543 [00:00<00:37, 337.34it/s]

Using BERT-base-cased tokenizer to align embeddings with PTB tokens


[aligning embeddings]: 100%|██████████| 12543/12543 [00:30<00:00, 417.41it/s]
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


...

[computing labels]: 100%|██████████| 2077/2077 [00:00<00:00, 7464.66it/s]
The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


Loading BERT Pretrained Embeddings from ../example/data/en_ewt-ud/en_ewt-ud-train.bert-base-layers.hdf5; using layer 9


[aligning embeddings]:   0%|          | 4/12543 [00:00<05:49, 35.90it/s]

Using BERT-base-cased tokenizer to align embeddings with PTB tokens


[aligning embeddings]:  49%|████▉     | 6151/12543 [01:39<02:39, 40.19it/s] 